In [1]:
import pandas as pd

# Đọc dữ liệu gốc và chuyển Timestamp thành datetime
df = pd.read_csv('./air_quality_weather_data.csv', parse_dates=['Timestamp'], dayfirst=True)

# Điều kiện lọc riêng cho từng trạm
condition_hcm = (
    (df['Name'] == 'HCM: Khu Liên cơ quan Bộ Tài Nguyên và Môi Trường - số 20 Đ. Lý Chính Thắng (KK)') &
    (df['Timestamp'] >= pd.Timestamp('2025-04-25'))
)

condition_quangbinh = (
    (df['Name'] == 'Quảng Bình: Khu kinh tế Hòn La (KK)') &
    (df['Timestamp'] >= pd.Timestamp('2025-04-22'))
)

# Các trạm còn lại giữ nguyên
condition_others = ~df['Name'].isin([
    'HCM: Khu Liên cơ quan Bộ Tài Nguyên và Môi Trường - số 20 Đ. Lý Chính Thắng (KK)',
    'Quảng Bình: Khu kinh tế Hòn La (KK)'
])

# Gộp điều kiện lọc và lọc dữ liệu
filtered_df = df[condition_hcm | condition_quangbinh | condition_others].copy()

# Định dạng lại cột Timestamp theo dạng dd/mm/yyyy HH:MM
filtered_df['Timestamp'] = filtered_df['Timestamp'].dt.strftime('%d/%m/%Y %H:%M')

# Ghi đè lại vào file
filtered_df.to_csv('./air_quality_weather_data_filtered.csv', index=False)

print("✅ Đã lọc và lưu lại file air_quality_weather_data_filtered.csv với định dạng thời gian chuẩn.")


✅ Đã lọc và lưu lại file air_quality_weather_data_filtered.csv với định dạng thời gian chuẩn.


In [2]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv('./air_quality_weather_data_filtered.csv', dayfirst=True)

# Chuyển Timestamp về đúng định dạng
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Các thông số cần kiểm tra
columns_to_check = ['PM2.5', 'PM10', 'CO', 'NO2', 'O3', 'SO2',
                    'Temperature', 'Humidity', 'Pressure', 'Wind Speed']

# Chuyển các cột về số
for col in columns_to_check:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Ngưỡng kiểm tra bất thường (vượt quá là vô lý)
thresholds = {
    'PM2.5': (0, 1000),        # µg/m³
    'PM10': (0, 1500),
    'CO': (0, 100),            # mg/m³
    'NO2': (0, 1000),
    'O3': (0, 1000),
    'SO2': (0, 1000),
    'Temperature': (-50, 60),  # độ C
    'Humidity': (0, 100),      # %
    'Pressure': (850, 1100),   # hPa
    'Wind Speed': (0, 60)      # m/s
}

# Lọc và in ra các giá trị bất thường
print("📌 Các giá trị vượt ngưỡng bất thường:\n")

for col, (min_val, max_val) in thresholds.items():
    outliers = df[(df[col] < min_val) | (df[col] > max_val)]
    if not outliers.empty:
        print(f"🔴 {col}: {len(outliers)} giá trị bất thường")
        print(outliers[['Timestamp', 'Name', col]].head(5), "\n")  # In ra 5 dòng đầu tiên
    else:
        print(f"✅ {col}: Không có giá trị bất thường")


📌 Các giá trị vượt ngưỡng bất thường:

✅ PM2.5: Không có giá trị bất thường
✅ PM10: Không có giá trị bất thường
✅ CO: Không có giá trị bất thường
✅ NO2: Không có giá trị bất thường
✅ O3: Không có giá trị bất thường
✅ SO2: Không có giá trị bất thường
✅ Temperature: Không có giá trị bất thường
✅ Humidity: Không có giá trị bất thường
✅ Pressure: Không có giá trị bất thường
✅ Wind Speed: Không có giá trị bất thường


In [3]:
import pandas as pd

df = pd.read_csv('./air_quality_weather_data_filtered.csv', dayfirst=True)

new_locations = [
    ("Hà Nội: ĐHBK cổng Parabol đường Giải Phóng (KK)", 21.0052, 105.8418),
    ("Hà Nội: 556 Nguyễn Văn Cừ (KK)", 21.0491, 105.8831),
    ("Hà Nội: Công viên Nhân Chính - Khuất Duy Tiến (KK)", 21.0031, 105.7947),
    ("HCM: Khu Liên cơ quan Bộ Tài Nguyên và Môi Trường - số 20 Đ. Lý Chính Thắng (KK)", 10.7823, 106.6834),
    ("HCM: Đ. Lê Hữu Kiều - P. Bình Trưng Tây - Quận 2 (Ngã ba Lê Hữu Kiểu và Trương Văn Bang) (KK)", 10.7823, 106.7528),
    ("Đà Nẵng: Khuôn viên trường ĐH sư phạm Đà Nẵng (KK)", 16.0622, 108.1594),
    ("Thái nguyên: Đường Hùng Vương - Tp Thái Nguyên (KK)", 21.59315, 105.8431),
    ("Phú Thọ: đường Hùng Vương - Tp Việt Trì (KK)", 21.33847, 105.3633),
    ("Bắc Giang: Khu liên cơ quan tỉnh Bắc Giang - P. Ngô Quyền - TP. Bắc Giang (KK)", 21.3015, 106.22603),
    ("Hà Nam: Công Viên Nam Cao - P.Quang Trung - TP. Phủ Lý (KK)", 20.536, 105.9165),
    ("Long An: UBND Tp Tân An - 76 Hùng Vương - P.2 (KK)", 10.5391, 106.4045),
    ("Bình Dương: số 593 Đại lộ Bình Dương, P. Hiệp Thành (KK)", 10.9923, 106.6577),
    ("Quảng Bình: Khu kinh tế Hòn La (KK)", 17.9329, 106.4966)
]

# Tạo dict để tìm nhanh
location_dict = {name: (lat, lon) for name, lat, lon in new_locations}

# Hàm tìm và cập nhật toạ độ
def update_coordinates(row):
    for loc_name in location_dict:
        if loc_name in row["Name"]:
            lat, lon = location_dict[loc_name]
            row["Latitude"] = lat
            row["Longitude"] = lon
            break
    return row

# Áp dụng cập nhật cho từng dòng
df = df.apply(update_coordinates, axis=1)

df.to_csv("air_quality_weather_data_filtered.csv", index=False)

In [4]:
import pandas as pd

# Đọc file CSV gốc
df = pd.read_csv('./air_quality_weather_data_filtered.csv', dayfirst=True)


# Thêm cột 'Source' vào đầu tiên với giá trị 'gov'
df.insert(0, 'Source', 'gov')

# Lưu lại file mới (hoặc ghi đè nếu muốn)
df.to_csv("air_quality_weather_data_filtered.csv", index=False)

In [5]:
import pandas as pd

# Đọc file CSV gốc
df = pd.read_csv('./air_quality_weather_data_filtered.csv', dayfirst=True)

# Tạo từ điển ánh xạ Name -> ID
name_to_id = {
    "Hà Nội: ĐHBK cổng Parabol đường Giải Phóng (KK)" :  31390903576425084107499649578,
    "Hà Nội: 556 Nguyễn Văn Cừ (KK)" : 28560877461938780203765592307,
    "Hà Nội: Công viên Nhân Chính - Khuất Duy Tiến (KK)" : 31390908889087377344742439468,
    "HCM: Khu Liên cơ quan Bộ Tài Nguyên và Môi Trường - số 20 Đ. Lý Chính Thắng (KK)" : 31390916083317566102523755051,
    "HCM: Đ. Lê Hữu Kiều - P. Bình Trưng Tây - Quận 2 (Ngã ba Lê Hữu Kiểu và Trương Văn Bang) (KK)" : 31390912357075263208060500522,
    "Đà Nẵng: Khuôn viên trường ĐH sư phạm Đà Nẵng (KK)" : 31388851800421997746903202346,
    "Thái nguyên: Đường Hùng Vương - Tp Thái Nguyên (KK)" : 29195707587706641566224751462,
    "Phú Thọ: đường Hùng Vương - Tp Việt Trì (KK)" : 28505268571336961948594948504,
    "Bắc Giang: Khu liên cơ quan tỉnh Bắc Giang - P. Ngô Quyền - TP. Bắc Giang (KK)" : 31387251434693138681789561386,
    "Hà Nam: Công Viên Nam Cao - P.Quang Trung - TP. Phủ Lý (KK)" : 31388883344354363840031242796,
    "Long An: UBND Tp Tân An - 76 Hùng Vương - P.2 (KK)" : 31390932574706768021562473002,
    "Bình Dương: số 593 Đại lộ Bình Dương, P. Hiệp Thành (KK)" : 31388839920718814259329251882,
    "Quảng Bình: Khu kinh tế Hòn La (KK)" : 29213751141295132066317063859,
}


# Tạo cột 'ID' từ Name
df.insert(1, 'ID', df['Name'].map(name_to_id))

# Lưu file kết quả
df.to_csv("air_quality_weather_data_filtered.csv", index=False)

In [8]:
import pandas as pd

# Đọc dữ liệu từ file gốc
df = pd.read_csv('air_quality_weather_data_filtered.csv')

# Loại bỏ dữ liệu của trạm Quảng Bình: KKT Hòn La (KK)
df_filtered = df[df['Name'] != 'Quảng Bình: KKT Hòn La (KK)']

# Ghi ra file mới (hoặc ghi đè lên file cũ nếu muốn)
df_filtered.to_csv('air_quality_weather_data_filtered.csv', index=False)



In [9]:
print(df_filtered['Name'].unique())


['Hà Nội: 556 Nguyễn Văn Cừ (KK)'
 'HCM: Đ. Lê Hữu Kiều - P. Bình Trưng Tây - Quận 2 (Ngã ba Lê Hữu Kiểu và Trương Văn Bang) (KK)'
 'Long An: UBND Tp Tân An - 76 Hùng Vương - P.2 (KK)'
 'Hà Nội: ĐHBK cổng Parabol đường Giải Phóng (KK)'
 'Hà Nội: Công viên Nhân Chính - Khuất Duy Tiến (KK)'
 'Đà Nẵng: Khuôn viên trường ĐH sư phạm Đà Nẵng (KK)'
 'Thái nguyên: Đường Hùng Vương - Tp Thái Nguyên (KK)'
 'Phú Thọ: đường Hùng Vương - Tp Việt Trì (KK)'
 'Bắc Giang: Khu liên cơ quan tỉnh Bắc Giang - P. Ngô Quyền - TP. Bắc Giang (KK)'
 'Hà Nam: Công Viên Nam Cao - P.Quang Trung - TP. Phủ Lý (KK)'
 'Bình Dương: số 593 Đại lộ Bình Dương, P. Hiệp Thành (KK)'
 'HCM: Khu Liên cơ quan Bộ Tài Nguyên và Môi Trường - số 20 Đ. Lý Chính Thắng (KK)']
